In [1]:
from collections import OrderedDict
from itertools import product
from pyCBT.data.providers.mt.pairs import get_mt_pairs
from highcharts import Highchart

import ffn

np = ffn.core.np
pd = ffn.data.pd
plt = ffn.core.plt

In [2]:
# define constants
# time frame in periods/minutes (because data is sampled in minutes)
TIME_FRAME = 30
# define target price
PRICE = "CLOSE"
# define volume column
VOLUME = "VOLUME"

pairs = get_mt_pairs()

PAIRS = pairs.keys()

# extract price for each instrument
price = pd.DataFrame(index=range(5*TIME_FRAME))
for pair_name in PAIRS:
    price[pair_name] = pairs[pair_name].filter(like=PRICE).values[-5*TIME_FRAME:]

# compute correlations
C = pd.DataFrame(data=np.nan, index=PAIRS, columns=PAIRS)
for i, j in product(range(len(PAIRS)), range(len(PAIRS))):
    price_i, price_j = price[PAIRS[i]][-TIME_FRAME:].values, price[PAIRS[j]][-TIME_FRAME:].values
    C.loc[PAIRS[i]][PAIRS[j]] = np.corrcoef(price_i, price_j)[1, 0]

In [88]:
def correlation_map(inp_pair, min_corr=None, max_corr=None):
    # filtering
    chart_dataset = C[inp_pair].dropna()
    mask = np.ones(chart_dataset.shape, dtype=np.bool)
    if min_corr is not None: mask = mask&(min_corr < abs(chart_dataset))
    if max_corr is not None: mask = mask&(max_corr > abs(chart_dataset))

    chart_dataset = chart_dataset.mask(~mask)
    chart_categories = list(chart_dataset.index.values)

    chart_options = {
        'chart': {
            'type': 'heatmap',
            'marginTop': 40,
            'marginBottom': 80,
            'plotBorderWidth': 1
        },
        'title': {
            'text': 'Correlations'
        },

        'xAxis': {
            'categories': inp_pair
        },

        'yAxis': {
            'categories': chart_categories,
            'title': {
                'text': None
            }
        },
        'colorAxis': {
            'min': -1,
            'max': +1,
    #         'minColor': '#86BAED',
            'maxColor': '#F46363',
    #         'stops': [[-1, '#86BAED'], [0, '#FFFFFF'], [+1, '#F46363']]
        },
        'legend': {
            'align': 'right',
            'layout': 'vertical',
            'margin': 0,
            'verticalAlign': 'top',
            'y': 25,
            'symbolHeight': 280
        },
        'tooltip': {
            'formatter': "function () {\
                return '<b>' + this.series.xAxis.categories[this.point.x] + '</b> correlates <br><b>' +\
                    this.point.value + '</b> with <br><b>' + this.series.yAxis.categories[this.point.y] + '</b>';\
            }"
        }
    }

    chart_dataset = [[i, j, round(chart_dataset.iloc[j][i], 2) if chart_dataset.iloc[j][i] != np.nan else None] for i, j in product(range(len(inp_pair)), range(len(chart_categories)))]

    chart = Highchart()
    chart.set_dict_options(chart_options)
    chart.add_data_set(chart_dataset, 'heatmap')
    return chart

In [89]:
correlation_map(inp_pair=["EURUSD", "EURGBP", "USDJPY"], min_corr=0.5)

In [67]:
def correlation_radar(inp_pair, min_corr=None, max_corr=None):
    # filtering
    chart_dataset = C[inp_pair].dropna()
    mask = np.zeros(chart_dataset.size, dtype=np.bool)
    if min_corr is not None: mask = mask|(min_corr > abs(chart_dataset))
    if max_corr is not None: mask = mask|(max_corr < abs(chart_dataset))

    chart_dataset = chart_dataset.mask(mask).dropna().drop(inp_pair)
    chart_categories = list(chart_dataset.index.values)
    chart_dataset = list(chart_dataset)

    chart_options = {

        'chart': {
            'polar': True,
            'type': 'line'
        },

        'title': {
            'text': inp_pair,
            'x': -80
        },

        'pane': {
            'size': '80%'
        },

        'xAxis': {
            'categories': chart_categories,
            'tickmarkPlacement': 'on',
            'lineWidth': 0
        },

        'yAxis': {
            'gridLineInterpolation': 'polygon',
            'lineWidth': 0,
            'min': 0,
            'max': 1
        },

        'tooltip': {
            'shared': True,
            'pointFormat': '<span style="color:{series.color}">{point.y:,.2f}<br/>'
        },

        'legend': {
            'align': 'right',
            'verticalAlign': 'top',
            'y': 70,
            'layout': 'vertical'
        }
    }

    dataset_1, dataset_2 = zip(*[(-round(corr, 2) if corr < 0.0 else 0.0, round(corr, 2) if corr > 0.0 else 0.0) for corr in chart_dataset])
    chart = Highchart()
    chart.set_dict_options(chart_options)
    chart.add_data_set(dataset_1, 'line', name="Negative Correlation", pointPlacement='off')
    chart.add_data_set(dataset_2, 'line', name="Positive Correlation", pointPlacement='off')
    return chart

In [28]:
charts = [correlation_radar(pair, min_corr=0.25, max_corr=1.0) for pair in ["EURUSD", "EURGBP", "USDJPY"]]
charts[0]

In [29]:
charts[1]

In [30]:
charts[2]